#### Return on Assets

In [3]:
# sp500_roa_power_fixed.py

import yfinance as yf
import pandas as pd
import numpy as np
import time
import requests
from io import StringIO

# 1) Get S&P500 list from Wikipedia (with headers to avoid 403)
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
response = requests.get(url, headers=headers)
if response.status_code != 200:
    raise Exception(f"Failed to fetch S&P 500 list. HTTP {response.status_code}")

sp500 = pd.read_html(StringIO(response.text))[0]
sp500.columns = [c.strip() for c in sp500.columns]
sp500 = sp500.rename(columns={"Symbol": "Ticker", "Security": "Full Name"})

# 2) Helper to find the right row label
def find_label(labels, candidates):
    for cand in candidates:
        mask = labels.str.contains(cand, case=False, na=False)
        if mask.any():
            return labels[mask].index[0]
    return None

# 3) Function to compute ROA Power
def collect_roa_power(ticker, n_years=5):
    try:
        t = yf.Ticker(ticker)
        fin = t.financials
        bs = t.balance_sheet
        if fin is None or fin.empty or bs is None or bs.empty:
            return None

        ni_label = find_label(fin.index.to_series(), ["net income", "netearn", "netprofit"])
        ta_label = find_label(bs.index.to_series(), ["total assets"])
        if ni_label is None or ta_label is None:
            return None

        ni = fin.loc[ni_label]
        ta = bs.loc[ta_label]

        # Compute ROA history
        roa = []
        for date in ni.index:
            n, a = ni.get(date), ta.get(date)
            if pd.notna(n) and pd.notna(a) and a != 0:
                roa.append(n / a)
            if len(roa) >= n_years:
                break

        if len(roa) < 2:
            return None

        # Compute year-to-year changes & power
        changes = np.diff(roa)
        sd = np.std(changes, ddof=0)
        power = float('inf') if sd == 0 else 1.0 / sd

        return {
            "Ticker": ticker,
            "Full Name": t.info.get("longName", np.nan),
            "ROA History": roa,
            "ROA Changes": changes.tolist(),
            "ROA Power": power
        }

    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        return None

# 4) Collect data for all tickers
results = []
for i, row in sp500.iterrows():
    ticker = row["Ticker"]
    rec = collect_roa_power(ticker)
    if rec:
        results.append(rec)
    print(f"[{i+1}/{len(sp500)}] Done: {ticker}")
    time.sleep(1)  # delay to avoid rate limit

# 5) Save and display
df = pd.DataFrame(results).set_index("Ticker")
print(df[["Full Name", "ROA Power"]])
df.to_csv("sp500_roa_power.csv", index=True)
print("\nSaved to sp500_roa_power.csv ✅")


[1/503] Done: MMM
[2/503] Done: AOS
[3/503] Done: ABT
[4/503] Done: ABBV
[5/503] Done: ACN
[6/503] Done: ADBE
[7/503] Done: AMD
[8/503] Done: AES
[9/503] Done: AFL
[10/503] Done: A
[11/503] Done: APD
[12/503] Done: ABNB
[13/503] Done: AKAM
[14/503] Done: ALB
[15/503] Done: ARE
[16/503] Done: ALGN
[17/503] Done: ALLE
[18/503] Done: LNT
[19/503] Done: ALL
[20/503] Done: GOOGL
[21/503] Done: GOOG
[22/503] Done: MO
[23/503] Done: AMZN
[24/503] Done: AMCR
[25/503] Done: AEE
[26/503] Done: AEP
[27/503] Done: AXP
[28/503] Done: AIG
[29/503] Done: AMT
[30/503] Done: AWK
[31/503] Done: AMP
[32/503] Done: AME
[33/503] Done: AMGN
[34/503] Done: APH
[35/503] Done: ADI
[36/503] Done: AON
[37/503] Done: APA
[38/503] Done: APO
[39/503] Done: AAPL
[40/503] Done: AMAT
[41/503] Done: APP
[42/503] Done: APTV
[43/503] Done: ACGL
[44/503] Done: ADM
[45/503] Done: ANET
[46/503] Done: AJG
[47/503] Done: AIZ
[48/503] Done: T
[49/503] Done: ATO
[50/503] Done: ADSK
[51/503] Done: ADP
[52/503] Done: AZO
[53/503]

In [ ]:
#ROA = pd.read_csv("sp500_roa_power.csv", index_col=0)
#print(ROA.sort_values(by="ROA", ascending=False))

KeyError: 'ROA'

### Return on Equity


In [4]:
# sp500_roe_power_fixed.py

import yfinance as yf
import pandas as pd
import numpy as np
import time
import requests
from io import StringIO

# 1) Get S&P500 list from Wikipedia (avoid 403 error)
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

response = requests.get(url, headers=headers)
if response.status_code != 200:
    raise Exception(f"Failed to fetch S&P 500 list. HTTP {response.status_code}")

sp500 = pd.read_html(StringIO(response.text))[0]
sp500.columns = [c.strip() for c in sp500.columns]
sp500 = sp500.rename(columns={"Symbol": "Ticker", "Security": "Full Name"})

# 2) Helper to find correct row label
def find_label(labels, candidates):
    for cand in candidates:
        mask = labels.str.contains(cand, case=False, na=False)
        if mask.any():
            return labels[mask].index[0]
    return None

# 3) Compute ROE Power
def collect_roe_power(ticker, n_years=5):
    try:
        t = yf.Ticker(ticker)
        fin = t.financials
        bs = t.balance_sheet

        if fin is None or fin.empty or bs is None or bs.empty:
            return None

        # Identify labels
        ni_label = find_label(fin.index.to_series(), ["net income", "netearn", "netprofit"])
        eq_label = find_label(bs.index.to_series(), ["stockholders equity", "total equity", "shareholders equity"])
        if ni_label is None or eq_label is None:
            return None

        ni = fin.loc[ni_label]
        eq = bs.loc[eq_label]

        # Build ROE history
        roe = []
        for date in ni.index:
            n, e = ni.get(date), eq.get(date)
            if pd.notna(n) and pd.notna(e) and e != 0:
                roe.append(n / e)
            if len(roe) >= n_years:
                break

        if len(roe) < 2:
            return None

        # Compute changes and power
        changes = np.diff(roe)
        sd = np.std(changes, ddof=0)
        power = float('inf') if sd == 0 else 1.0 / sd

        return {
            "Ticker": ticker,
            "Full Name": t.info.get("longName", np.nan),
            "ROE History": roe,
            "ROE Changes": changes.tolist(),
            "ROE Power": power
        }

    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        return None

# 4) Loop through S&P 500 list
results = []
for i, row in sp500.iterrows():
    ticker = row["Ticker"]
    rec = collect_roe_power(ticker)
    if rec:
        results.append(rec)
    print(f"[{i+1}/{len(sp500)}] Done: {ticker}")
    time.sleep(1)  # avoid throttling from Yahoo

# 5) Save results
ROE = pd.DataFrame(results).set_index("Ticker")
print(ROE[["Full Name", "ROE Power"]])
ROE.to_csv("sp500_roe_power.csv", index=True)
print("\nSaved to sp500_roe_power.csv ✅")


[1/503] Done: MMM
[2/503] Done: AOS
[3/503] Done: ABT
[4/503] Done: ABBV
[5/503] Done: ACN
[6/503] Done: ADBE
[7/503] Done: AMD
[8/503] Done: AES
[9/503] Done: AFL
[10/503] Done: A
[11/503] Done: APD
[12/503] Done: ABNB
[13/503] Done: AKAM
[14/503] Done: ALB
[15/503] Done: ARE
[16/503] Done: ALGN
[17/503] Done: ALLE
[18/503] Done: LNT
[19/503] Done: ALL
[20/503] Done: GOOGL
[21/503] Done: GOOG
[22/503] Done: MO
[23/503] Done: AMZN
[24/503] Done: AMCR
[25/503] Done: AEE
[26/503] Done: AEP
[27/503] Done: AXP
[28/503] Done: AIG
[29/503] Done: AMT
[30/503] Done: AWK
[31/503] Done: AMP
[32/503] Done: AME
[33/503] Done: AMGN
[34/503] Done: APH
[35/503] Done: ADI
[36/503] Done: AON
[37/503] Done: APA
[38/503] Done: APO
[39/503] Done: AAPL
[40/503] Done: AMAT
[41/503] Done: APP
[42/503] Done: APTV
[43/503] Done: ACGL
[44/503] Done: ADM
[45/503] Done: ANET
[46/503] Done: AJG
[47/503] Done: AIZ
[48/503] Done: T
[49/503] Done: ATO
[50/503] Done: ADSK
[51/503] Done: ADP
[52/503] Done: AZO
[53/503]

In [ ]:
print(ROE.sort_values(by="ROE Power", ascending=False))

                                  Full Name  \
Ticker                                        
LNT              Alliant Energy Corporation   
ELV                   Elevance Health, Inc.   
AEE                      Ameren Corporation   
CPRT                           Copart, Inc.   
XEL                        Xcel Energy Inc.   
...                                     ...   
MSI                Motorola Solutions, Inc.   
MTCH                      Match Group, Inc.   
NCLH    Norwegian Cruise Line Holdings Ltd.   
MTD       Mettler-Toledo International Inc.   
MCK                    McKesson Corporation   

                                              ROE History  \
Ticker                                                      
LNT     [0.09851513420902341, 0.10373321528700015, 0.1...   
ELV     [0.1447416192666102, 0.1523177123085534, 0.162...   
AEE     [0.09757305596830114, 0.10150674068199841, 0.1...   
CPRT    [0.181156034992506, 0.20672290661785336, 0.235...   
XEL     [0.09917016699

### Return on Invested Capital

In [5]:
# sp500_roic_power_fixed.py

import yfinance as yf
import pandas as pd
import numpy as np
import time
import requests
from io import StringIO

# 1) Get S&P500 list from Wikipedia (avoid HTTP 403)
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

response = requests.get(url, headers=headers)
if response.status_code != 200:
    raise Exception(f"Failed to fetch S&P 500 list. HTTP {response.status_code}")

sp500 = pd.read_html(StringIO(response.text))[0]
sp500.columns = [c.strip() for c in sp500.columns]
sp500 = sp500.rename(columns={"Symbol": "Ticker", "Security": "Full Name"})

# 2) Helper to find correct row label
def find_label(labels, candidates):
    for cand in candidates:
        mask = labels.str.contains(cand, case=False, na=False)
        if mask.any():
            return labels[mask].index[0]
    return None

# 3) Compute ROIC Power
def collect_roic_power(ticker, n_years=5):
    try:
        t = yf.Ticker(ticker)
        fin = t.financials
        bs = t.balance_sheet

        if fin is None or fin.empty or bs is None or bs.empty:
            return None

        # Find key rows
        op_label = find_label(fin.index.to_series(), ["operating income", "ebit", "oper income"])
        ta_label = find_label(bs.index.to_series(), ["total assets"])
        cl_label = find_label(bs.index.to_series(), ["current liabilities"])

        if op_label is None or ta_label is None or cl_label is None:
            return None

        op = fin.loc[op_label]
        ta = bs.loc[ta_label]
        cl = bs.loc[cl_label]

        # Compute ROIC history: EBIT / (Total Assets - Current Liabilities)
        roic = []
        for date in op.index:
            o, a, c = op.get(date), ta.get(date), cl.get(date)
            dc = a - c if pd.notna(a) and pd.notna(c) else None
            if pd.notna(o) and dc and dc != 0:
                roic.append(o / dc)
            if len(roic) >= n_years:
                break

        if len(roic) < 2:
            return None

        # Compute year-to-year changes and ROIC Power
        changes = np.diff(roic)
        sd = np.std(changes, ddof=0)
        power = float("inf") if sd == 0 else 1.0 / sd

        return {
            "Ticker": ticker,
            "Full Name": t.info.get("longName", np.nan),
            "ROIC History": roic,
            "ROIC Changes": changes.tolist(),
            "ROIC Power": power
        }

    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        return None

# 4) Loop & collect
results = []
for i, row in sp500.iterrows():
    ticker = row["Ticker"]
    rec = collect_roic_power(ticker)
    if rec:
        results.append(rec)
    print(f"[{i+1}/{len(sp500)}] Done: {ticker}")
    time.sleep(1)  # polite delay to avoid API throttling

# 5) Save and display
df = pd.DataFrame(results).set_index("Ticker")
print(df[["Full Name", "ROIC Power"]])
df.to_csv("sp500_roic_power.csv", index=True)
print("\nSaved to sp500_roic_power.csv ✅")


[1/503] Done: MMM
[2/503] Done: AOS
[3/503] Done: ABT
[4/503] Done: ABBV
[5/503] Done: ACN
[6/503] Done: ADBE
[7/503] Done: AMD
[8/503] Done: AES
[9/503] Done: AFL
[10/503] Done: A
[11/503] Done: APD
[12/503] Done: ABNB
[13/503] Done: AKAM
[14/503] Done: ALB
[15/503] Done: ARE
[16/503] Done: ALGN
[17/503] Done: ALLE
[18/503] Done: LNT
[19/503] Done: ALL
[20/503] Done: GOOGL
[21/503] Done: GOOG
[22/503] Done: MO
[23/503] Done: AMZN
[24/503] Done: AMCR
[25/503] Done: AEE
[26/503] Done: AEP
[27/503] Done: AXP
[28/503] Done: AIG
[29/503] Done: AMT
[30/503] Done: AWK
[31/503] Done: AMP
[32/503] Done: AME
[33/503] Done: AMGN
[34/503] Done: APH
[35/503] Done: ADI
[36/503] Done: AON
[37/503] Done: APA
[38/503] Done: APO
[39/503] Done: AAPL
[40/503] Done: AMAT
[41/503] Done: APP
[42/503] Done: APTV
[43/503] Done: ACGL
[44/503] Done: ADM
[45/503] Done: ANET
[46/503] Done: AJG
[47/503] Done: AIZ
[48/503] Done: T
[49/503] Done: ATO
[50/503] Done: ADSK
[51/503] Done: ADP
[52/503] Done: AZO
[53/503]